In [ ]:
!pip install datasets transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("tweet_eval", "emotion")  # "emotion" is the chosen task


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1421 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

In [ ]:
# Print some sample entries
print(dataset['train'][0])
print(dataset['train'][1])

# Check the distribution of labels
from collections import Counter

counter = Counter(dataset['train']['label'])
print(counter)


{'text': "“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry", 'label': 2}
{'text': "My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs", 'label': 0}
Counter({0: 1400, 3: 855, 1: 708, 2: 294})


In [ ]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3257 [00:00<?, ? examples/s]

Map:   0%|          | 0/1421 [00:00<?, ? examples/s]

Map:   0%|          | 0/374 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(tokenized_datasets['train'], batch_size=16, shuffle=True)
val_loader = DataLoader(tokenized_datasets['validation'], batch_size=16)
test_loader = DataLoader(tokenized_datasets['test'], batch_size=16)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report

# Since you already have the dataset loaded, we will extract the texts and labels directly.
texts = dataset['train']['text']
labels = dataset['train']['label']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.15, random_state=42)

# Create a machine learning pipeline
pipeline = make_pipeline(TfidfVectorizer(), LogisticRegression(max_iter=2000))

# Train the model
pipeline.fit(X_train, y_train)

# Predict on the test set
predictions = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


Accuracy: 0.6462167689161554
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.94      0.73       191
           1       0.85      0.44      0.58       124
           2       0.86      0.12      0.21        49
           3       0.66      0.60      0.63       125

    accuracy                           0.65       489
   macro avg       0.74      0.53      0.54       489
weighted avg       0.70      0.65      0.61       489



In [ ]:
# Improving model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# Data Preparation
texts = dataset['train']['text']
labels = dataset['train']['label']
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.15, random_state=42)

# Pipeline Creation with Parameter Tuning
pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1, 2), max_features=5000),
    LogisticRegression(max_iter=2000)
)

# Define a grid of parameters
param_grid = {
    'logisticregression__C': [0.1, 1, 10],
    'logisticregression__penalty': ['l1', 'l2']
}

# Grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best model
print("Best parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# Predict and Evaluate
predictions = best_model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Improved Accuracy:", accuracy)
print("Improved Classification Report:\n", report)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
 

Best parameters: {'logisticregression__C': 10, 'logisticregression__penalty': 'l2'}
Improved Accuracy: 0.6952965235173824
Improved Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.85      0.76       191
           1       0.80      0.63      0.70       124
           2       0.70      0.29      0.41        49
           3       0.63      0.68      0.66       125

    accuracy                           0.70       489
   macro avg       0.70      0.61      0.63       489
weighted avg       0.70      0.70      0.68       489



In [ ]:
import joblib

# Save the model to disk
joblib.dump(best_model, 'text_classification_model.joblib')

print("Model saved successfully!")


Model saved successfully!


In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
print(os.listdir())


['.config', 'text_classification_model.joblib', 'sample_data']


In [ ]:
from google.colab import files
files.download('text_classification_model.joblib')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>